# Problem Definition

## Dataset:
Consists of all even numbers between zero and 2^n as binary representations

Example number 56 is 0111000.

This will allow us to test the performance of the generator easily.

## Generator

takes in random noise and learns to produce only even numbers.

Noise Variants:

*   Random interger in the range (0,10) for every bit
*   A single random interger in the range (0,10) 
*   A single random floating value in the range (0,1) for every bit




---




# Convert a number to binary

In [2]:
import math
def get_binary_representation(number, max_number = 128):
    number_of_bits = int(math.log(max_number, 2))
    code = "{0:0" + str(number_of_bits) + "b}"
    string_representation = code.format(number)
    return [int(char) for char in string_representation]


get_binary_representation(1)

[0, 0, 0, 0, 0, 0, 1]

# Generate data for training

In [4]:
import math
import torch
import numpy as np
import random

# Generates #batch_size even numbers in the range 0 to max_number
def generate_training_data(max_number = 128, batch_size = 16):
  
    data = []
    labels = [1] * batch_size
    for x in range(batch_size):
        data.append(get_binary_representation(random.randrange(0, max_number-1, 2)))

    return torch.tensor(labels).float(), torch.tensor(data).float()

generate_training_data()

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([[1., 1., 0., 1., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 1., 1., 0.],
         [1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0.],
         [0., 0., 0., 1., 1., 0., 0.],
         [0., 0., 1., 1., 0., 1., 0.],
         [0., 1., 0., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0.],
         [1., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0.]]))

# Generator and Discriminator

In [5]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_length):
        super(Generator, self).__init__()
        self.linear_layer = nn.Linear(input_length, input_length)
        self.activation = nn.Sigmoid()
  
    def forward(self, x):
        return self.activation(self.linear_layer(x)).squeeze(1)

class Discriminator(nn.Module):
    def __init__(self, input_length):
        super(Discriminator, self).__init__()
        self.linear_layer = nn.Linear(input_length, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.linear_layer(x)).squeeze(1)

# Training 

In [12]:

def train(max_number = 128, batch_size = 16, training_steps = 1000):

    input_length = int(math.log(max_number, 2))

    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    generator_optimizer = torch.optim.Adam(generator.parameters(), lr = training_parameters["generator_lr"])
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = training_parameters["discriminator_lr"])

    loss = nn.BCELoss()

    for training_step in range(training_steps):
    
    # Generator

        generator_optimizer.zero_grad()

        # Creating noise of #input_length bits
        noise = torch.randint(0, 10, size=(batch_size, input_length)).float()
        generated_data = generator(noise)

        true_labels, true_data = generate_training_data(max_number, batch_size=batch_size)

        discriminator_output_on_generated_data = discriminator(generated_data).view(batch_size)
        generator_loss = loss(discriminator_output_on_generated_data, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        # Discriminator
        '''
        It is important to zero.grad() the gradients for discriminator since in the above steps gradients were accumulated 
        but we don't want to update weights of discriminator in that step.

        '''
        discriminator_optimizer.zero_grad()

        discriminator_output_for_true_data = discriminator(true_data)
        true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)


        '''
         It is important to note that when passing in the generated data we want to detach the gradients. 
         We do this because we are not training the generator we are just focused on the discriminator.
        '''

        discriminator_output_for_generated_data = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(
            discriminator_output_for_generated_data, torch.zeros(batch_size)
        )
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2

        discriminator_loss.backward()
        discriminator_optimizer.step()

        if training_step % 5000 == 0:
            print("Training Steps Completed: ", training_step)

    return generator, discriminator



# Training Parameters

In [13]:
training_parameters = {
    "max_number": 128,
    "batch_size": 16,
    "generator_lr": 0.001,
    "discriminator_lr": 0.001,
}
generator, discriminator = train(training_steps = 50000)

Training Steps Completed:  0
Training Steps Completed:  5000
Training Steps Completed:  10000
Training Steps Completed:  15000
Training Steps Completed:  20000
Training Steps Completed:  25000
Training Steps Completed:  30000
Training Steps Completed:  35000
Training Steps Completed:  40000
Training Steps Completed:  45000


# Let's see what the generator outputs

In [14]:
with torch.no_grad():
    input_length = int(math.log(training_parameters["max_number"], 2))
    noise = torch.randint(0, 10, size=(2, input_length)).float()
    print(generator(noise))

tensor([[6.0135e-02, 9.3795e-01, 9.9777e-01, 1.6846e-04, 9.9968e-01, 4.7197e-04,
         1.5523e-24],
        [6.3910e-03, 1.3177e-01, 9.9992e-01, 7.6263e-05, 9.9939e-01, 2.8300e-06,
         1.5848e-23]])


# So, it generates a float number for every bit - it is basically trying to replicate the training data. 

# We take a threshold and assign it as 0 OR 1.

In [9]:
def convert_float_representation_to_binary_representation(float_representations, threshold = 0.5):
    binary_representations = []
    for float_representation in float_representations:
        binary_representations.append((float_representation >= threshold).int())
     
    return binary_representations

# Convert binary to decimal


In [10]:
def convert_binary_to_decimal(n):
    n = n.tolist()
    n = [str(i) for i in n]
    n = "".join(n)
    return int(n,2)
convert_binary_to_decimal(torch.tensor([0,0,1,0])) 

2

In [15]:
with torch.no_grad():
    input_length = int(math.log(training_parameters["max_number"], 2))
    noise = torch.randint(0, 10, size=( training_parameters["batch_size"], input_length)).float()
    binary_reps = convert_float_representation_to_binary_representation(generator(noise))
    for i in range(len(binary_reps)):
        binary_rep = binary_reps[i]
        print("when noise is: ",(noise[i]))
        # print(binary_rep)
        print("Generated number: ", convert_binary_to_decimal(binary_rep))
        print("---")



when noise is:  tensor([9., 8., 5., 3., 4., 6., 2.])
Generated number:  52
---
when noise is:  tensor([8., 7., 7., 0., 2., 1., 0.])
Generated number:  20
---
when noise is:  tensor([0., 4., 1., 6., 2., 8., 7.])
Generated number:  116
---
when noise is:  tensor([9., 9., 3., 7., 6., 4., 6.])
Generated number:  52
---
when noise is:  tensor([3., 2., 2., 6., 0., 5., 6.])
Generated number:  20
---
when noise is:  tensor([3., 8., 8., 5., 1., 2., 5.])
Generated number:  52
---
when noise is:  tensor([0., 5., 4., 9., 5., 0., 8.])
Generated number:  116
---
when noise is:  tensor([9., 4., 8., 1., 9., 5., 3.])
Generated number:  52
---
when noise is:  tensor([4., 2., 7., 8., 5., 1., 2.])
Generated number:  20
---
when noise is:  tensor([5., 1., 4., 7., 7., 5., 6.])
Generated number:  52
---
when noise is:  tensor([7., 8., 4., 9., 6., 5., 9.])
Generated number:  52
---
when noise is:  tensor([0., 5., 0., 5., 1., 9., 8.])
Generated number:  116
---
when noise is:  tensor([8., 8., 2., 6., 2., 9., 5

---
---
# Let's try with a single digit noise but in that case there are only 10 possibilities of noise and hence in the output





In [16]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_length):
        super(Generator, self).__init__()
        self.linear_layer = nn.Linear(1, input_length)
        self.activation = nn.Sigmoid()
  
    def forward(self, x):
        return self.activation(self.linear_layer(x)).squeeze(1)

class Discriminator(nn.Module):
    def __init__(self, input_length):
        super(Discriminator, self).__init__()
        self.linear_layer = nn.Linear(input_length, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.linear_layer(x)).squeeze(1)


def train(max_number = 128, batch_size = 16, training_steps = 1000):

    input_length = int(math.log(max_number, 2))

    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    generator_optimizer = torch.optim.Adam(generator.parameters(), lr = training_parameters["generator_lr"])
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = training_parameters["discriminator_lr"])

    loss = nn.BCELoss()

    for training_step in range(training_steps):
    
    # Generator

        generator_optimizer.zero_grad()

        # Creating noise of #input_length bits
        noise = torch.randint(0, 10, size=(batch_size, 1)).float()
        generated_data = generator(noise)

        true_labels, true_data = generate_training_data(max_number, batch_size=batch_size)

        discriminator_output_on_generated_data = discriminator(generated_data).view(batch_size)
        generator_loss = loss(discriminator_output_on_generated_data, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        # Discriminator
        '''
        It is important to zero.grad() the gradients for discriminator since in the above steps gradients were accumulated 
        but we don't want to update weights of discriminator in that step.

        '''
        discriminator_optimizer.zero_grad()

        discriminator_output_for_true_data = discriminator(true_data)
        true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)


        '''
         It is important to note that when passing in the generated data we want to detach the gradients. 
         We do this because we are not training the generator we are just focused on the discriminator.
        '''

        discriminator_output_for_generated_data = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(
            discriminator_output_for_generated_data, torch.zeros(batch_size)
        )
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2

        discriminator_loss.backward()
        discriminator_optimizer.step()

        if training_step % 5000 == 0:
            print("Training Steps Completed: ",training_step)

    return generator, discriminator

training_parameters = {
    "max_number": 128,
    "batch_size": 16,
    "generator_lr": 0.001,
    "discriminator_lr": 0.001,
}
generator, discriminator = train(training_steps = 50000)



Training Steps Completed:  0
Training Steps Completed:  5000
Training Steps Completed:  10000
Training Steps Completed:  15000
Training Steps Completed:  20000
Training Steps Completed:  25000
Training Steps Completed:  30000
Training Steps Completed:  35000
Training Steps Completed:  40000
Training Steps Completed:  45000


In [17]:
with torch.no_grad():
    input_length = int(math.log(training_parameters["max_number"], 2))
    noise = torch.tensor(range(0,10)).float().unsqueeze(1)
    binary_reps = convert_float_representation_to_binary_representation(generator(noise))
    for i in range(len(binary_reps)):
        binary_rep = binary_reps[i]
        print("when noise is: ",int(noise[i][0].item()))
        # print(binary_rep)
        print("Generated number: ", convert_binary_to_decimal(binary_rep))
        print("---")


when noise is:  0
Generated number:  92
---
when noise is:  1
Generated number:  92
---
when noise is:  2
Generated number:  108
---
when noise is:  3
Generated number:  108
---
when noise is:  4
Generated number:  108
---
when noise is:  5
Generated number:  108
---
when noise is:  6
Generated number:  108
---
when noise is:  7
Generated number:  108
---
when noise is:  8
Generated number:  108
---
when noise is:  9
Generated number:  108
---


# Noise is a floating point number in the range (0,1) for every bit

In [19]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_length):
        super(Generator, self).__init__()
        self.linear_layer = nn.Linear(input_length, input_length)
        self.activation = nn.Sigmoid()
  
    def forward(self, x):
        return self.activation(self.linear_layer(x)).squeeze(1)

class Discriminator(nn.Module):
    def __init__(self, input_length):
        super(Discriminator, self).__init__()
        self.linear_layer = nn.Linear(input_length, 1)
        self.activation = nn.Sigmoid()
  
    def forward(self, x):
        return self.activation(self.linear_layer(x)).squeeze(1)


def train(max_number = 128, batch_size = 16, training_steps = 1000):

    input_length = int(math.log(max_number, 2))

    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    generator_optimizer = torch.optim.Adam(generator.parameters(), lr = training_parameters["generator_lr"])
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = training_parameters["discriminator_lr"])

    loss = nn.BCELoss()

    for training_step in range(training_steps):
    
        # Generator

        generator_optimizer.zero_grad()

        # Creating noise of #input_length bits
        # noise = torch.randint(0, 10, size=(batch_size, input_length)).float()
        noise = torch.rand(batch_size, input_length)
        generated_data = generator(noise)

        true_labels, true_data = generate_training_data(max_number, batch_size=batch_size)

        discriminator_output_on_generated_data = discriminator(generated_data).view(batch_size)
        generator_loss = loss(discriminator_output_on_generated_data, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        # Discriminator
        '''
        It is important to zero.grad() the gradients for discriminator since in the above steps gradients were accumulated 
        but we don't want to update weights of discriminator in that step.

        '''
        discriminator_optimizer.zero_grad()

        discriminator_output_for_true_data = discriminator(true_data)
        true_discriminator_loss = loss(discriminator_output_for_true_data, true_labels)


        '''
         It is important to note that when passing in the generated data we want to detach the gradients. 
         We do this because we are not training the generator we are just focused on the discriminator.
        '''

        discriminator_output_for_generated_data = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(
            discriminator_output_for_generated_data, torch.zeros(batch_size)
        )
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2

        discriminator_loss.backward()
        discriminator_optimizer.step()

        if training_step % 5000 == 0:
            print("Training Steps Completed: ", training_step)

    return generator, discriminator


training_parameters = {
    "max_number": 128,
    "batch_size": 16,
    "generator_lr": 0.001,
    "discriminator_lr": 0.001,
}
generator, discriminator = train(training_steps = 50000)


Training Steps Completed:  0
Training Steps Completed:  5000
Training Steps Completed:  10000
Training Steps Completed:  15000
Training Steps Completed:  20000
Training Steps Completed:  25000
Training Steps Completed:  30000
Training Steps Completed:  35000
Training Steps Completed:  40000
Training Steps Completed:  45000


In [20]:
with torch.no_grad():
    input_length = int(math.log(training_parameters["max_number"], 2))
    # noise = torch.randint(0, 10, size=( training_parameters["batch_size"], input_length)).float()
    noise = torch.rand(training_parameters["batch_size"], input_length)

    binary_reps = convert_float_representation_to_binary_representation(generator(noise))
    for i in range(len(binary_reps)):
        binary_rep = binary_reps[i]
        print("when noise is: ",(noise[i]))
        # print(binary_rep)
        print("Generated number: ", convert_binary_to_decimal(binary_rep))
        print("---")



when noise is:  tensor([0.6225, 0.0369, 0.7624, 0.1954, 0.8584, 0.1222, 0.3718])
Generated number:  98
---
when noise is:  tensor([0.9042, 0.2575, 0.4305, 0.1611, 0.4935, 0.0673, 0.9583])
Generated number:  66
---
when noise is:  tensor([0.6216, 0.8725, 0.2764, 0.2445, 0.5246, 0.4450, 0.2369])
Generated number:  66
---
when noise is:  tensor([0.9486, 0.5285, 0.8126, 0.5933, 0.2355, 0.3959, 0.4888])
Generated number:  98
---
when noise is:  tensor([0.7753, 0.5874, 0.7034, 0.6238, 0.4979, 0.8341, 0.1090])
Generated number:  98
---
when noise is:  tensor([0.3145, 0.1065, 0.9520, 0.9779, 0.0270, 0.0555, 0.0814])
Generated number:  98
---
when noise is:  tensor([0.7220, 0.7835, 0.7316, 0.4158, 0.6531, 0.6581, 0.5803])
Generated number:  66
---
when noise is:  tensor([0.0975, 0.4882, 0.4739, 0.5245, 0.9468, 0.8146, 0.9106])
Generated number:  66
---
when noise is:  tensor([0.4927, 0.4466, 0.7148, 0.0619, 0.3439, 0.9166, 0.0020])
Generated number:  98
---
when noise is:  tensor([0.9709, 0.796